In [4]:

import os
import pandas as pd
import json

from model import num2cate, generate_samples, generate_model_samples, ModelGene, findKeyAttrs, FindGroups, get_numAttrs, find_rules
from model.samples import DataGene
from model.data_encoder import DataEncoder
from joblib import dump, load

store_path = '../../front/src/testdata/'


def get_rules(dataset_name, protect_attr='',model_name=None):
    
    model_name = '{}_{}'.format(dataset_name, model_name)
    sample_path = os.path.join(store_path, '{}_samplesR.csv'.format(model_name))
    model_samples = pd.read_csv(sample_path)
    rules = find_rules(model_samples, minimum_support=5, min_len=1, protect_attr = protect_attr, target_attr='class', elift_th=[1, 1])

    
    rules.to_json(store_path + '{}_rules.json'.format(model_name),orient='records')

    
def get_samples(dataset_name, protect_attr):
    """
    train model on the training data,
    return the generated samples based on the training data
    """

    sample_num = 1000 # number of generated data 
    dataset_path = '../server/{}.csv'.format(dataset_name)
    data = pd.read_csv(dataset_path)
    
#     for i,dat in data.iterrows():
#         data.at[i,'class'] = 'H' if data.at[i,'class']=='H' else 'L'
    
    # generate samples
    samplesInit = generate_samples(data, sample_num)

    model=['lr','knn','xgb']
    

#     # find key_attrs
#     key_attrs = findKeyAttrs(num2cate(data), protect_attr)
#     f = open(store_path + '{}_key.json'.format(dataset_name),'w')
#     json.dump(key_attrs, f)
    
    for var in model:
        
        # set name
        model_name = dataset_name+"_"+var
        model_gene = ModelGene(model_name)
    
        # train model
        model, encoder, score = model_gene.fit_model(num2cate(data))
        
#         # general samples
#         num_samples, cate_samples = generate_model_samples(samplesInit, model, encoder) 
        
#         # add the ID col 
#         cate_samples.insert(loc=0, column='id', value= cate_samples.index)
#         num_samples.insert(loc=0, column='id', value= num_samples.index)
        
#         # save mdeol & samples to cache
#         dataOut = pd.concat([num_samples,cate_samples])

#         samples_path = os.path.join(store_path, '{}_samplesR.json'.format(model_name))
#         cate_samples.to_json(samples_path, orient='records')

#         samples_path = os.path.join(store_path, '{}_samplesR.csv'.format(model_name))
#         cate_samples.to_csv(samples_path, index=False)

#         samples_path = os.path.join(store_path, '{}_samples.json'.format(model_name))
#         dataOut.to_json(samples_path, orient='records')

#         samples_path = os.path.join(store_path, '{}_samples.csv'.format(model_name))
#         dataOut.to_csv(samples_path, index=False)
        
        print(dataset_name + ' ' + var + ' accuracy: ' + str(score))
        
    print(dataset_name + ' all done')

# get_samples('bank','maritary')
get_samples('adult', 'sex')

TypeError: transform() takes 2 positional arguments but 3 were given

In [ ]:
# get_rules('dataTest','sex= Female', 'knn')
# # get_rules('dataTest','sex= Fema le', 'lr')
# get_rules('dataTest','sex= Female', 'knn')
# # get_rules('dataTest','sex= Fema le', 'lr')
# dataSets = ['dataTest','bank']
# models = ['xgb','knn','lr']
# print('start')
# for dataSet in dataSets:
#     for model in models:
#         sample_name = dataSet + '_' + model + '_samples_concat.json'
#         rule_name = dataSet + '_' + model + '_rules.json'
#         samples = pd.read_json(sample_name)

#         # add item id to rules
#         def item_within_rule(item, rule_context):
#             for attr_val in rule_context:
#                 attr, val = attr_val.split('=')
#                 if not item[attr] == val :
#                     return False
#             return True

#         rules = pd.read_json(rule_name)
#         rules['items'] = ''
#         for idx, rule in rules.iterrows():
#             rules.at[idx, 'items'] = [sample['id'] for i,sample in samples.loc[1000:1999].iterrows() if item_within_rule(sample, rule["antecedent"])] 
#         rules.to_json(rule_name, orient='records')

#         print(dataSet+'_'+model+' has done')

# print('All finished')

In [13]:
import pandas as pd


df = pd.read_csv('german_credit.csv')

df['gender'] = ''
df['status'] = ''

for i,d in df.iterrows():
    df.at[i,'gender'] = df.at[i,'personal_status'].split(' ')[0]
    df.at[i,'status'] = df.at[i,'personal_status'].split(' ')[1]
df.drop(['personal_status'],axis=1)
df.to_csv('german_credit.csv',index=False)

In [22]:
'''explore the number of samples for each rules'''
import pandas as pd

df = pd.read_json('academic_knn_rules.json')
df = pd.DataFrame(df)
for i,d in df.iterrows():
    print(df.at[i,'antecedent'])
    if(df.at[i,'antecedent']==['StudentAbsenceDays=Under-7', 'raisedhands=0<x<25', 'Discussion=1<x<25']):
        print(df.at[i,'id'])

['Discussion=50<x<74', 'Topic=Biology']
['StudentAbsenceDays=Under-7', 'SectionID=C', 'Discussion=50<x<74']
['VisITedResources=x>74', 'GradeID=G-10']
['StageID=lowerlevel', 'Discussion=25<x<50', 'raisedhands=x>75']
['SectionID=A', 'Topic=Science']
['Relation=Father', 'StageID=lowerlevel', 'Discussion=50<x<74']
['StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'PlaceofBirth=Syria']
['SectionID=A', 'Discussion=25<x<50', 'VisITedResources=24<x<49']
['VisITedResources=x>74', 'GradeID=G-12']
['Semester=F', 'ParentschoolSatisfaction=Bad', 'SectionID=A', 'VisITedResources=0<x<24']
['ParentschoolSatisfaction=Bad', 'SectionID=A', 'Discussion=50<x<74']
['StageID=lowerlevel', 'SectionID=A', 'AnnouncementsView=49<x<73']
['SectionID=A', 'Topic=French']
['Semester=F', 'SectionID=A', 'Topic=Science']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'Relation=Father', 'Discussion=50<x<74']
['Semester=F', 'StudentAbsenceDays=Under-7', 'SectionID=A', 'VisITedResources=0<x<24']
['S

['Semester=F', 'Relation=Father', 'StageID=lowerlevel', 'VisITedResources=0<x<24']
['StudentAbsenceDays=Under-7', 'SectionID=A', 'VisITedResources=0<x<24']
['SectionID=C', 'Topic=French']
['Relation=Father', 'VisITedResources=0<x<24', 'AnnouncementsView=x>73']
['SectionID=B', 'NationalITy=SaudiArabia']
['ParentAnsweringSurvey=No', 'Topic=Science']
['StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'raisedhands=x>75']
['SectionID=A', 'VisITedResources=x>74', 'raisedhands=x>75']
['ParentschoolSatisfaction=Bad', 'VisITedResources=x>74', 'raisedhands=50<x<75']
['Semester=F', 'ParentAnsweringSurvey=No', 'GradeID=G-07']
['Semester=F', 'StudentAbsenceDays=Under-7', 'StageID=MiddleSchool', 'VisITedResources=x>74']
['ParentAnsweringSurvey=No', 'GradeID=G-10']
['Semester=F', 'Topic=French']
['SectionID=C', 'StageID=MiddleSchool', 'AnnouncementsView=x>73']
['Semester=F', 'StageID=lowerlevel', 'SectionID=A', 'raisedhands=x>75']
['Semester=F', 'StudentAbsenceDays=Under-7', 'SectionID=C', 'S

['StudentAbsenceDays=Under-7', 'StageID=lowerlevel', 'AnnouncementsView=0<x<24', 'VisITedResources=49<x<74']
['StudentAbsenceDays=Under-7', 'Relation=Father', 'ParentAnsweringSurvey=No', 'StageID=MiddleSchool', 'VisITedResources=x>74']
['ParentAnsweringSurvey=No', 'SectionID=B', 'VisITedResources=x>74', 'Discussion=1<x<25']
['StageID=lowerlevel', 'SectionID=C', 'raisedhands=0<x<25']
['ParentschoolSatisfaction=Bad', 'Discussion=25<x<50', 'NationalITy=Syria']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'Discussion=25<x<50', 'AnnouncementsView=x>73']
['ParentAnsweringSurvey=No', 'Discussion=25<x<50', 'NationalITy=Syria']
['StageID=MiddleSchool', 'Discussion=25<x<50', 'NationalITy=Palestine']
['Relation=Father', 'SectionID=B', 'StageID=HighSchool', 'Discussion=25<x<50']
['Semester=F', 'raisedhands=0<x<25', 'AnnouncementsView=0<x<24', 'VisITedResources=49<x<74']
['Relation=Father', 'ParentAnsweringSurvey=No', 'SectionID=A', 'raisedhands=50<x<75']
['StageID=HighSchool', 'D

['Semester=F', 'ParentAnsweringSurvey=No', 'AnnouncementsView=0<x<24', 'NationalITy=Jordan']
['Relation=Father', 'AnnouncementsView=49<x<73', 'GradeID=G-07']
['SectionID=C', 'Discussion=1<x<25', 'AnnouncementsView=49<x<73', 'VisITedResources=24<x<49']
['Semester=F', 'SectionID=C', 'Discussion=25<x<50', 'raisedhands=50<x<75']
['ParentschoolSatisfaction=Bad', 'Relation=Father', 'ParentAnsweringSurvey=No', 'NationalITy=SaudiArabia']
['StudentAbsenceDays=Under-7', 'Relation=Father', 'SectionID=A', 'GradeID=G-12']
['StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'SectionID=B', 'VisITedResources=x>74', 'Discussion=1<x<25']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'StageID=MiddleSchool', 'Discussion=1<x<25']
['SectionID=C', 'PlaceofBirth=Palestine']
['raisedhands=0<x<25', 'Discussion=50<x<74', 'GradeID=G-07']
['StudentAbsenceDays=Under-7', 'VisITedResources=0<x<24', 'Discussion=1<x<25', 'raisedhands=50<x<75']
['Relation=Father', 'Par

['VisITedResources=0<x<24']
['ParentschoolSatisfaction=Bad', 'SectionID=A', 'raisedhands=25<x<50']
['Relation=Father', 'StageID=lowerlevel', 'AnnouncementsView=24<x<49']
['ParentschoolSatisfaction=Bad', 'SectionID=C', 'VisITedResources=0<x<24']
['StudentAbsenceDays=Under-7', 'Relation=Father', 'SectionID=A', 'VisITedResources=x>74']
['StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'SectionID=A', 'AnnouncementsView=24<x<49']
['Semester=F', 'ParentAnsweringSurvey=No', 'SectionID=C', 'raisedhands=25<x<50']
['StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'SectionID=B', 'AnnouncementsView=24<x<49']
['Semester=F', 'ParentschoolSatisfaction=Bad', 'StageID=MiddleSchool', 'VisITedResources=x>74']
['Discussion=25<x<50', 'AnnouncementsView=x>73']
['StageID=MiddleSchool', 'VisITedResources=0<x<24', 'Discussion=25<x<50']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'AnnouncementsView=0<x<24']
['Relation=Father', 'SectionID=A', 'raisedhands=25<x<50']
['Semester

['SectionID=C', 'Discussion=x>74']
['SectionID=A', 'Topic=Geology']
['AnnouncementsView=0<x<24', 'raisedhands=x>75', 'Discussion=50<x<74']
['SectionID=A', 'NationalITy=Tunis']
['Semester=F', 'StageID=lowerlevel', 'VisITedResources=0<x<24']
['ParentschoolSatisfaction=Bad', 'VisITedResources=0<x<24', 'Discussion=1<x<25']
['raisedhands=50<x<75']
['ParentAnsweringSurvey=No', 'SectionID=C']
['Relation=Father', 'StageID=MiddleSchool', 'Discussion=50<x<74']
['ParentschoolSatisfaction=Bad', 'SectionID=C', 'raisedhands=50<x<75']
['AnnouncementsView=x>73', 'GradeID=G-12']
['StageID=MiddleSchool', 'VisITedResources=0<x<24']
['Semester=F', 'StageID=HighSchool', 'raisedhands=0<x<25']
['ParentAnsweringSurvey=No', 'VisITedResources=0<x<24']
['StudentAbsenceDays=Under-7', 'AnnouncementsView=24<x<49', 'raisedhands=25<x<50']
['StudentAbsenceDays=Under-7', 'Relation=Father', 'raisedhands=0<x<25']
['GradeID=G-08']
['VisITedResources=x>74', 'AnnouncementsView=x>73']
['ParentschoolSatisfaction=Bad', 'Studen

['ParentschoolSatisfaction=Bad', 'SectionID=C', 'VisITedResources=49<x<74']
['StageID=lowerlevel', 'SectionID=A', 'AnnouncementsView=24<x<49', 'Discussion=1<x<25']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'StageID=MiddleSchool', 'VisITedResources=x>74', 'AnnouncementsView=24<x<49']
['StageID=HighSchool', 'Discussion=50<x<74', 'Topic=Spanish']
['ParentschoolSatisfaction=Bad', 'StageID=HighSchool', 'NationalITy=Iran']
['StageID=HighSchool', 'GradeID=G-08']
['ParentAnsweringSurvey=No', 'StageID=HighSchool', 'PlaceofBirth=Tunis']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'SectionID=B', 'AnnouncementsView=0<x<24']
['StudentAbsenceDays=Under-7', 'Discussion=1<x<25', 'VisITedResources=49<x<74']
['SectionID=A', 'VisITedResources=49<x<74', 'NationalITy=USA']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'SectionID=A', 'GradeID=G-06']
['Relation=Father', 'AnnouncementsView=x>73', 'PlaceofBirth=venzuela']
['ra

['SectionID=C', 'VisITedResources=x>74', 'Discussion=1<x<25']
['AnnouncementsView=x>73', 'GradeID=G-08']
['SectionID=A', 'Topic=Math']
['StageID=lowerlevel', 'GradeID=G-08']
['Relation=Father', 'StageID=MiddleSchool', 'GradeID=G-02']
['VisITedResources=x>74', 'Discussion=1<x<25']
['StageID=lowerlevel', 'Topic=Arabic']
['SectionID=A', 'StageID=HighSchool', 'AnnouncementsView=24<x<49']
['VisITedResources=49<x<74', 'raisedhands=50<x<75']
['Relation=Father', 'ParentAnsweringSurvey=No', 'SectionID=C', 'AnnouncementsView=0<x<24']
['ParentschoolSatisfaction=Bad', 'SectionID=C', 'PlaceofBirth=KuwaIT']
['ParentschoolSatisfaction=Bad', 'StudentAbsenceDays=Under-7', 'SectionID=C', 'StageID=MiddleSchool', 'Discussion=25<x<50']
['StudentAbsenceDays=Under-7', 'ParentAnsweringSurvey=No', 'raisedhands=0<x<25', 'Discussion=x>74']
['SectionID=B', 'VisITedResources=0<x<24', 'AnnouncementsView=x>73']
['ParentschoolSatisfaction=Bad', 'ParentAnsweringSurvey=No', 'StageID=HighSchool', 'Discussion=25<x<50']
[

['StudentAbsenceDays=Under-7', 'raisedhands=50<x<75', 'PlaceofBirth=Morocco']
['ParentAnsweringSurvey=No', 'raisedhands=50<x<75', 'Topic=Math']
['Discussion=25<x<50', 'AnnouncementsView=49<x<73', 'Topic=IT']
['ParentschoolSatisfaction=Bad', 'SectionID=B', 'AnnouncementsView=24<x<49', 'raisedhands=25<x<50']
['StudentAbsenceDays=Under-7', 'Relation=Father', 'StageID=lowerlevel', 'VisITedResources=x>74', 'AnnouncementsView=49<x<73']
['AnnouncementsView=0<x<24', 'Discussion=x>74', 'GradeID=G-08']
['StageID=MiddleSchool', 'raisedhands=0<x<25', 'NationalITy=USA']
['Relation=Father', 'StageID=lowerlevel', 'AnnouncementsView=24<x<49', 'VisITedResources=24<x<49']
['AnnouncementsView=24<x<49', 'raisedhands=x>75', 'Discussion=x>74']
['Semester=F', 'StageID=lowerlevel', 'SectionID=C', 'AnnouncementsView=49<x<73']
['Semester=F', 'StageID=lowerlevel', 'NationalITy=Tunis']
['ParentschoolSatisfaction=Bad', 'Discussion=1<x<25', 'PlaceofBirth=lebanon']
['ParentschoolSatisfaction=Bad', 'SectionID=C', 'St